# Racetrack

Exercise 5.10: Racetrack (programming) Consider driving a race car around a turn like those shown in
Figure 5.5. You want to go as fast as possible, but not so fast as to run off the track. In our simplified
racetrack, the car is at one of a discrete set of grid positions, the cells in the diagram. The velocity is
also discrete, a number of grid cells moved horizontally and vertically per time step. The actions are
increments to the velocity components. Each may be changed by +1, −1, or 0 in each step, for a total
of nine (3 × 3) actions. Both velocity components are restricted to be nonnegative and less than 5,
and they cannot both be zero except at the starting line. Each episode begins in one of the randomly
selected start states with both velocity components zero and ends when the car crosses the finish line.
The rewards are −1 for each step until the car crosses the finish line. If the car hits the track boundary,
it is moved back to a random position on the starting line, both velocity components are reduced to
zero, and the episode continues. Before updating the car’s location at each time step, check to see if
the projected path of the car intersects the track boundary. If it intersects the finish line, the episode
ends; if it intersects anywhere else, the car is considered to have hit the track boundary and is sent
back to the starting line. To make the task more challenging, with probability 0.1 at each time step
the velocity increments are both zero, independently of the intended increments. Apply a Monte Carlo control method to this task to compute the optimal policy from each starting state. Exhibit several
trajectories following the optimal policy (but turn the noise off for these trajectories).

To simplify the problem we consider a smooth racetrack, i.e. a rectangle with the lower right quarter missing
The rectangle is 10 x 20 (so there are 5 starting positions)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Enviroment

In [2]:
class Racetrack():
    
    def __init__(self):
        self._size = (20,40)
        self.reset()
        
    def step(self, user_action):
        action = self._preprocess_action(user_action)
        self._update_velocity(action)
        self._update_position()
        reward = -1
        return self._convert_position(), reward, self._done
    
    def reset(self):
        self._velocity = [0,0]
        self._done = False
        self._position = [np.random.randint(self._size[0] // 2), 0]
        return self._convert_position()
    
    def _update_position(self):
        # simple checking, not totally correct
        self._position[0] += self._velocity[0]
        self._position[1] += self._velocity[1]
        if self._check_for_finish():
            self._done = True
        elif self._check_for_boundary_hit():
            self.reset()
        return
        
    def _check_for_finish(self):
        return self._position[0] >= self._size[0] and self._position[1] >= self._size[1] / 2
    
    def _check_for_boundary_hit(self):
        hit_boundary = False
        if self._position[0] >= self._size[0] // 2 and self._position[1] <= self._size[1] // 2:
            hit_boundary = True
        elif self._position[0] >= self._size[0] or self._position[1] >= self._size[1]:
            hit_boundary = True
        return hit_boundary
        
    def _update_velocity(self, action):
        tmp1 = self._velocity[0] + action[0]
        tmp2 = self._velocity[1] + action[1]
        self._velocity[0] = min(max(0, tmp1),5)
        self._velocity[1] = min(max(0, tmp2),5)
    
    def _preprocess_action(self, user_action):
        action = self._convert_action(user_action)
        action = self._disturb_action(action)
        return action
    
    def _disturb_action(self, action):
        if np.random.rand() < 0.0:
            action = [0,0]
        return action
    
    def _convert_position(self):
        return self._position[1] * self._size[0] + self._position[0]
    
    def _convert_action(self, action):
        return [(action % 3)-1, (action // 3)-1]
    
    

## General Functions

In [3]:
def create_history_mc_episode(q, env, eps):
    history = []
    state = env.reset()
    done = False
    while not done:
        action = choose_action_eps_greedy(q, state, eps)
        next_state, reward, done = env.step(action)
        history.append((state, action, reward))
        state = next_state
    return history
        
def choose_action_eps_greedy(q, state, eps):
    if np.random.rand() < eps:
        return np.random.randint(9)
    else:
        return np.argmax(q[state,:])
    
def create_first_visit_entries(history):
    first_visit = -np.ones(800)
    for idx, entry in enumerate(history):
        if first_visit[entry[0]] == -1:
            first_visit[entry[0]] = idx
    return first_visit

## On-policy first visit MC control

In [ ]:
def first_visit_on_policy_mc_control(n_episode):
    q = np.zeros((800,9))
    eps = 1
    env = Racetrack()
    for episode in range(n_episode):
        if episode % 5000 == 0:
            eps /= 2
        history = create_history_mc_episode(q, env, eps)
        print(len(history), eps)
        q = update_q_function(q, history)
        
def update_q_function(q, history):
    g = 0
    first_visit = create_first_visit_entries(history)
    for idx, entry in enumerate(history[::-1]):
        converted_idx = len(history)-idx-1
        g += entry[2]
        if first_visit[entry[0]] == converted_idx:
            q[entry[0], entry[1]] += 0.01 * (g - q[entry[0], entry[1]])
    return q

In [ ]:
first_visit_on_policy_mc_control(50000)

## Ordinary Importance Sampling

In [6]:
def ordinary_importance_sampling_mc_control(n_episode):
    q = np.zeros((800,9))
    eps = 0.01
    env = Racetrack()
    for episode in range(n_episode):
        history = create_history_mc_episode(q, env, eps)
        print(len(history))
        q = update_q_function_ordinary_is(q, history, eps)
        
def update_q_function_ordinary_is(q, history, eps):
    g = 0
    q_old = q.copy()
    first_visit = create_first_visit_entries(history)
    ratio = 1
    for idx, entry in enumerate(history[::-1]):
        ratio *= compute_importance_sampling_ratio_factor(entry[0], entry[1], q_old, eps)
        if ratio == 0:
            print('Ratio is zero in step ', idx)
            break
        converted_idx = len(history)-idx-1
        g += entry[2]
        if first_visit[entry[0]] == converted_idx:
            q[entry[0], entry[1]] += 0.1 * ratio * (g - q[entry[0], entry[1]])
    return q
            
def compute_importance_sampling_ratio_factor(state, action, q, eps):
    ratio = 0
    if np.argmax(q[state,:]) == action:
        # prob of choosing the greedy action given the eps-greedy policy
        ratio = 1 / (1 - eps + eps/9)
    return ratio

In [7]:
ordinary_importance_sampling_mc_control(1000)

4484455
Ratio is zero in step  0
1253202
Ratio is zero in step  0
1434563
Ratio is zero in step  0
1591783
Ratio is zero in step  0
732342
Ratio is zero in step  0
2294400
Ratio is zero in step  0
82359
Ratio is zero in step  0
447104
Ratio is zero in step  0
945146
Ratio is zero in step  0
2929891
Ratio is zero in step  0
1863284
Ratio is zero in step  0
2210049
Ratio is zero in step  0
371058
Ratio is zero in step  0
531041
Ratio is zero in step  0
632431
Ratio is zero in step  0
843995
Ratio is zero in step  0
2509317
Ratio is zero in step  0
1269138
Ratio is zero in step  0
1628586
Ratio is zero in step  1
888338
Ratio is zero in step  0
161479
Ratio is zero in step  1
671224
Ratio is zero in step  0
2219878
Ratio is zero in step  0
190682
Ratio is zero in step  0
129423
Ratio is zero in step  0
3093660
Ratio is zero in step  0
3246885
Ratio is zero in step  0
1508954
Ratio is zero in step  0
1146522
Ratio is zero in step  1
673888
Ratio is zero in step  0
142460
Ratio is zero in s

KeyboardInterrupt: 

## Weighted Importance Sampling